In [1]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report 
from sklearn.metrics import accuracy_score 
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection  import RFE

In [2]:

# df = pd.read_csv("merged_dataset.csv", engine="python", sep=",")
# X = df.drop(columns=["Is.Fraudulent"]).copy()
# y = df["Is.Fraudulent"].copy()

# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# ValidationData = pd.concat([X_val, y_val], axis=1)  
# ValidationData.to_csv("ValidationData.csv", index=False)

# X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=42, stratify=y_train)

# y_test = pd.DataFrame(y_test, columns=["Is.Fraudulent"])
# y_train = pd.DataFrame(y_train, columns=["Is.Fraudulent"])


# TestData = pd.concat([X_test, y_test], axis=1)  
# TestData.to_csv("TestData.csv", index=False)

# TrainData = pd.concat([X_train, y_train], axis=1)  
# TrainData.to_csv("TrainData.csv", index=False)


In [3]:
class TimeTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self  
    def transform(self, X):
        df = X.copy()  
        if "Transaction.Date" not in df.columns: 
            raise ValueError("What are you doing man?")
        df["Transaction.Date"] = pd.to_datetime(df["Transaction.Date"], format = "ISO8601")
                    
        df["day"] = df["Transaction.Date"].dt.day.astype(float)
        
        df["month"] = df["Transaction.Date"].dt.month.astype(float)
        df["month_sin"] = np.sin(2 * np.pi * df["month"] / 12)
        df["month_cos"] = np.cos(2 * np.pi * df["month"] / 12)
        df["month_angle"]=np.arctan2(df["month_sin"],df["month_cos"]) 
        
        if "Transaction.Hour" in df.columns:
            df["hour_sin"] = np.sin(2 * np.pi * df["Transaction.Hour"] / 24)
            df["hour_cos"] = np.cos(2 * np.pi * df["Transaction.Hour"] / 24)
            df["hour_angle"] = np.arctan2(df["hour_sin"], df["hour_cos"])
        else:
            raise ValueError("What are you doing man? Time")
        
        df["Transaction.Weekday"] = df["Transaction.Date"].dt.weekday + 1
        df["FirstPartMonth"]=df["day"].apply(lambda x: 1 if x<=12 else 0) 
            
        weekDaysEncoded=pd.get_dummies(df["Transaction.Weekday"], dtype=int) 
        
        result=pd.concat([df[["month_angle", "hour_angle","FirstPartMonth"]], weekDaysEncoded], axis=1)
        
        return result.to_numpy()
    

In [4]:
class AgeTransfomer(BaseEstimator, TransformerMixin): 

    def __init__(self): 
        pass   
    def fit(self, X, y=None):
        return self  
    def transform(self, X):
        df=X.copy() 
        if "Customer.Age" not in df.columns: 
            raise ValueError("What are you doing man? Age")
        df["Is.Minor"]=df["Customer.Age"].apply(lambda x : 1 if x<18 else 0) 
        df["Is.Senior"]=df["Customer.Age"].apply(lambda x : 1 if x>60 else 0) 
        return pd.concat([df[["Is.Minor"]], df[["Is.Senior"]]], axis = 1).to_numpy()
        

In [5]:
class SexTransformer(BaseEstimator, TransformerMixin): 
    def __init__(self): 
         pass    
    def fit(self, X, y=None):
        return self  
    def transform(self, X):
        df=X.copy() 
        if "sex" not in df.columns: 
            raise ValueError("What are you doing man? Sex")
        df["male"]=df["sex"].apply(lambda x : 1 if x=="M" else 0)
        return df[["male"]].to_numpy() 

In [6]:
class BinaryPassthroughTransformer(BaseEstimator, TransformerMixin): 
    def __init__(self): 
        pass   
    def fit(self, X, y=None):
        return self  
    def transform(self, X):  
        df=X.copy()
        return df[["Address.Match"]].to_numpy() 

In [7]:
class HighAmountTransformer(BaseEstimator, TransformerMixin): 
    def __init__(self): 
        pass   
    def fit(self, X, y=None):
        return self  
    def transform(self, X):
        df = X.copy()
        if "Transaction.Amount" not in df.columns: 
            raise ValueError("What are you doing man? Amount")
        HighAmountInt=df["Transaction.Amount"].quantile(0.95)
        df["Is.HighAmount"]=df["Transaction.Amount"].apply(lambda x : 1 if x>=HighAmountInt else 0) 
        return df[["Is.HighAmount"]].to_numpy() 

In [8]:
def KCrossData():
    df = df=pd.read_csv("data/TrainData.csv")
    X = df.drop(columns=["Is.Fraudulent"]).copy()
    y = df["Is.Fraudulent"].copy()
    return X,y
def GetScalePosWeight(y): 
    return y.sum()/y.shape[0]

In [9]:
def PipelineModel(model,Numerical=['Transaction.Amount', 'Customer.Age','Account.Age.Days','Quantity'],
                    CatBasic=["Payment.Method",'browser','Product.Category','Device.Used','source']):

    column_transformer=PipeLineColumnTransformer(Numerical,CatBasic)
    selector=CreateFeatureSelector()
    classifier_pipeline=Pipeline([
    ('preprocessor', column_transformer),  
   # ('smote', SMOTE(sampling_strategy=0.1,random_state=42)),
   # ('featureselection',selector),
    ('model',model)
     ])
    return classifier_pipeline


## TO DISCUSS
def CreateFeatureSelector(): 
    log_clf = LogisticRegression(C=0.1, class_weight="balanced",  penalty='l1', 
    solver='liblinear', random_state=42)
    selector = RFE(estimator=log_clf, n_features_to_select=30, step=1)
    return selector

    
def PipeLineColumnTransformer(Numerical=['Transaction.Amount', 'Customer.Age','Account.Age.Days','Quantity'],
                    CatBasic=["Payment.Method",'browser','Product.Category','Device.Used','source']): 
    column_transformer = ColumnTransformer([
        ('time_features', TimeTransformer(),["Transaction.Date","Transaction.Hour"]), 
        ("high_amount",HighAmountTransformer(),["Transaction.Amount"]),
        ("numerical",StandardScaler(),Numerical), 
        ("age",AgeTransfomer(),["Customer.Age"]),
        ("sex",SexTransformer(),["sex"]),
        ("AddressMatch",BinaryPassthroughTransformer(),["Address.Match"]),
        ("catBasic", OneHotEncoder(drop='if_binary' , handle_unknown='ignore'),CatBasic)], remainder='drop')
    return column_transformer



def PredictionQualityInfo(y_pred,y_test):
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["Not Fraud", "Fraud"], yticklabels=["Not Fraud", "Fraud"])
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix")
    plt.show()
    print("Classification Report:\n", classification_report(y_test, y_pred))

    
def FitPredictResult(model,X_train,X_test,y_train,y_test):
    classifier=PipelineModel(model)
    classifier.fit(X_train,y_train) 
    y_pred=classifier.predict(X_test) 
    PredictionQualityInfo(y_pred,y_test) 

In [10]:
x,y=KCrossData()
clf=PipeLineColumnTransformer() 
clf.fit(x) 
r=clf.transform(x) 
r[1,:] 

array([ 1.57079633, -2.61799388,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        , -0.39246666, -1.68561393, -1.05328069, -0.70881097,
        0.        ,  0.        ,  1.        ,  1.        ,  1.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  1.        ,  0.        ,  0.        ])

In [11]:
#(147000, 39) After transformation